In [ ]:
import torch
import torch.nn as nn
 
class GCNLayer(nn.Module):
 
    def __init__(self,c_in,c_out):
        """
        Inputs:
        :param c_in: 输入特征
        :param c_out: 输出特征
        """
        super().__init__()
        self.projection = nn.Linear(c_in,c_out); #线性层
        
    def forward(self,node_feats,adj_matrix):
        """
        输入
        :param node_feats: 节点特征表示，大小为[batch_size,num_nodes,c_in]
        :param adj_matrix: 邻接矩阵：[batch_size,num_nodes,num_nodes]
        :return:
        """
        num_neighbors = adj_matrix.sum(dim=-1,keepdims=True) #各节点的邻居数
        node_feats = self.projection(node_feats)# 将特征转化为消息
        #各邻居节点消息求和并求平均
        node_feats = torch.bmm(adj_matrix,node_feats)
        node_feats = node_feats / num_neighbors
        
        return node_feats




In [ ]:
node_feats = torch.arange(8,
dtype=torch.float32).view(1,4,2)
adj_matrix = torch.Tensor([[[1,1,0,0],
            [1,1,1,1],
            [0,1,1,1],
            [0,1,1,1]]])
print("节点特征：\n",node_feats)
print("添加自链接的邻接矩阵：\n",adj_matrix)

layer = GCNLayer(c_in=2, c_out=2)
# 初始化权重矩阵
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

# 将节点特征和添加自连接的邻接矩阵输入 GCN 层
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("节点特征:\n", node_feats)
print("添加自连接的邻接矩阵:\n", adj_matrix)
print("节点输出特征:\n", out_feats)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
 
class GATLayer(nn.Module):
 
    def __init__(self,c_in,c_out,
                num_heads=1, concat_heads=True, alpha=0.2):
        """
        :param c_in: 输入特征维度
        :param c_out: 输出特征维度
        :param num_heads: 多头的数量
        :param concat_heads: 是否拼接多头计算的结果
        :param alpha: LeakyReLU的参数
        :return:
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = num_heads
        if self.concat_heads:
            assert c_out % num_heads ==0,"输出特征数必须是头数的倍数！"
            c_out = c_out // num_heads
 
        #参数
        self.projection = nn.Linear(c_in,c_out*num_heads) #有几个头，就需要将c_out扩充几倍
        self.a = nn.Parameter(torch.Tensor(num_heads,2*c_out)) #用于计算注意力的参数，由于对两节点拼接后的向量进行操作，所以2*c_out
        self.leakrelu = nn.LeakyReLU(alpha) #激活层
 
        #参数初始化
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
 
    def forward(self,node_feats,adj_matrix,print_attn_probs=False):
        """
        输入：
        :param self:
        :param node_feats: 节点的特征表示
        :param adj_matrix: 邻接矩阵
        :param print_attn_probs: 是否打印注意力
        :return:
        """
        batch_size,num_nodes = node_feats.size(0),node_feats.size(1)

        #将节点初始输入进行权重运算
        node_feats = self.projection(node_feats)
        #扩展出多头数量的维度
        node_feats = node_feats.view(batch_size,num_nodes,self.num_heads,-1)

        # 获取所有顶点对拼接而成的特征向量 a_input
        edges = adj_matrix.nonzero(as_tuple=False)  # 返回所有邻接矩阵中值不为 0 的 index，即所有连接的边对应的两个顶点
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)  # 将所有 batch_size 的节点拼接

        edge_indices_row = edges[:, 0] * batch_size + edges[:, 1]  # 获取边对应的第一个顶点 index
        edge_indices_col = edges[:, 0] * batch_size + edges[:, 2]  # 获取边对应的第二个顶点 index

        a_input = torch.cat([
        torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0), # 基于边对应的第一个顶点的 index 获取其特征值
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)  # 基于边对应的第二个顶点的 index 获取其特征值
        ], dim=-1)  # 两者拼接

        # 基于权重 a 进行注意力计算
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a)
        # LeakyReLU 计算
        attn_logits = self.leakrelu(attn_logits)

        # 将注意力权转换为矩阵的形式
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape + (self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[..., None].repeat(1, 1, 1, self.num_heads) == 1] = attn_logits.reshape(-1)

        # Softmax 计算转换为概率
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("注意力权重:\n", attn_probs.permute(0, 3, 1, 2))
        # 对每个节点进行注意力加权相加的计算
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)

        # 根据是否将多头的计算结果拼接与否进行不同操作
        if self.concat_heads:  # 拼接
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:  # 平均
            node_feats = node_feats.mean(dim=2)

        return node_feats 

In [ ]:
layer = GATLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                                    [1, 1, 1, 1],
                                    [0, 1, 1, 1],
                                    [0, 1, 1, 1]]])
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)


print("节点特征:\n", node_feats)
print("添加自连接的邻接矩阵:\n", adj_matrix)
print("节点输出特征:\n", out_feats)

In [ ]:
import torch
from torch_geometric.data import Data
# 基于节点的index表示边
#[0,1,1,2]表示出发的节点index
#[1,0,2,1]表示到达index
edge_index = torch.tensor([[0,1,1,2],
                           [1,0,2,1]],dtype=torch.long)
x = torch.tensor([[-1],[0],[1]],dtype=torch.float)#节点的特征矩阵，有3个节点，特征维度为1
 
data = Data(x=x,edge_index = edge_index) #初始化图
print(data)#查看图属性

In [ ]:
from torch_geometric.datasets import TUDataset
 
# 导入数据集
dataset = TUDataset(
    # 指定数据集的存储位置
    # 如果指定位置没有相应的数据集
    # PyG会自动下载
    root='ENZYMES/',
    # 要使用的数据集
    name='ENZYMES',
)
# 数据集的长度
print(len(dataset))
# 数据集的类别数
print(dataset.num_classes)
# 数据集中节点属性向量的维度
print(dataset.num_node_features)
# 600个图，我们可以根据索引选择要使用哪个图
data = dataset[100]
print(data)
# 随机打乱数据集
dataset = dataset.shuffle()

In [ ]:
from torch_geometric.data import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True) # 批次大小为 32，并且数据的顺序随机打乱

for batch in loader:
    print("一批数据：",batch)
    print("一批数据量：",batch.num_graphs)

In [ ]:
from torch_geometric.datasets import ShapeNet
dataset = ShapeNet(root='Airplane', categories=['Airplane'])
print(dataset[0])

In [ ]:
import torch_geometric.transforms as T
dataset = ShapeNet(root='Airplane', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6)) # 进行 KNN 聚类操作

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
 
def visualize_networkx(graph, color):
    plt.figure(figsize=(8,8)) # 设定图画区域大小
    nx.draw_networkx(graph, with_labels=False,node_color=color) # 画图
    plt.show()

In [ ]:
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx
 
dataset = KarateClub()[0] # 取图数据集
G = to_networkx(dataset,to_undirected=True) # 转化为 networkx
visualize_networkx(G, color=dataset.y) # 画图